In [2]:
import pandas as pd
import features_processing_utils as fpu
import hopsworks
import numpy as np

Log into Hopsworks

In [3]:
project = hopsworks.login()
# util.purge_project(proj)

2025-01-07 23:40:21,645 INFO: Initializing external client
2025-01-07 23:40:21,646 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 23:40:23,878 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079


In [4]:
fs = project.get_feature_store()

In [5]:
gossipcop_fake = pd.read_csv("FakeNewsNet/gossipcop_fake.csv")

Didn't run it for 2800 to 3000 index

In [6]:
# gossipcop_new_features = fpu.complete_processing(gossipcop_fake, 1, "gossipcop_news_posts.csv", "gossipcop_news_features.csv", 3300, 3500)

In [7]:
# gossipcop_new_features.info()

In [8]:
gossipcop_real = pd.read_csv("FakeNewsNet/gossipcop_real.csv")

In [9]:
gossipcop_new_features = fpu.complete_processing(gossipcop_real, 0, "gossipcop_new_posts.csv", "gossipcop_new_features.csv", 2000, 2200)

 47%|████▋     | 94/200 [03:37<01:47,  1.01s/it]

In [10]:
gossipcop_new_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    13 non-null     object 
 1   average followers          13 non-null     float64
 2   average follows            13 non-null     float64
 3   repost total               13 non-null     int64  
 4   post total                 13 non-null     int64  
 5   repost percentage          13 non-null     float64
 6   label                      13 non-null     int64  
 7   average repost             13 non-null     float64
 8   average favorite           13 non-null     float64
 9   news lifetime              13 non-null     float64
 10  nb users 10 hours          13 non-null     int64  
 11  average time difference    13 non-null     float64
 12  retweet percentage 1 hour  13 non-null     float64
dtypes: float64(8), int64(4), object(1)
memory usage: 1.4

In [36]:
#gossicop_fake_features = fpu.complete_processing(gossipcop_fake, 1, "gossipcop_fake_posts_4.csv", "gossipcop_fake_features_4.csv", 3000, 4000)
# gossicop_fake_features.info()

In [37]:
# fpu.complete_processing(gossipcop_fake, 1, "gossipcop_fake_posts_5.csv", "gossipcop_fake_features_5.csv", 4000, 5000)

In [38]:
# fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_3.csv", "gossipcop_real_features_3.csv", 2000, 3000)

In [39]:
# fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_4.csv", "gossipcop_real_features_4.csv", 3000, 4000)

In [40]:
# gossicop_real_features = fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_5.csv", "gossipcop_real_features_5.csv", 4000)
# gossicop_real_features.info()

In [41]:
gossipcop_features = pd.read_csv("data/gossipcop_features.csv")
gossipcop_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    374 non-null    object 
 1   average_followers          374 non-null    float64
 2   average_follows            374 non-null    float64
 3   repost_total               374 non-null    int64  
 4   post_total                 374 non-null    int64  
 5   repost_percentage          374 non-null    float64
 6   label                      374 non-null    int64  
 7   average_repost             374 non-null    float64
 8   average_favorite           374 non-null    float64
 9   news_lifetime              374 non-null    float64
 10  nb_users_10_hours          374 non-null    int64  
 11  average_time_difference    374 non-null    float64
 12  retweet_percentage_1_hour  374 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 3

In [42]:
# change names of columns to add underscore
gossipcop_features.columns = gossipcop_features.columns.str.replace(' ', '_')
gossipcop_new_features.columns = gossipcop_new_features.columns.str.replace(' ', '_')

In [43]:
gossipcop_features = pd.concat([gossipcop_features, gossipcop_new_features], ignore_index=True)

In [44]:
gossipcop_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    403 non-null    object 
 1   average_followers          403 non-null    float64
 2   average_follows            403 non-null    float64
 3   repost_total               403 non-null    int64  
 4   post_total                 403 non-null    int64  
 5   repost_percentage          403 non-null    float64
 6   label                      403 non-null    int64  
 7   average_repost             403 non-null    float64
 8   average_favorite           403 non-null    float64
 9   news_lifetime              403 non-null    float64
 10  nb_users_10_hours          403 non-null    int64  
 11  average_time_difference    403 non-null    float64
 12  retweet_percentage_1_hour  403 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 4

In [45]:
gossipcop_features.to_csv("data/gossipcop_features.csv", index=False)

In [46]:
news_propagation_fg = fs.get_feature_group(
    name="news_propagation",
    version=1,
)

In [47]:
historical_features = news_propagation_fg.select_all().read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.05s) 


In [48]:
historical_features = pd.concat([historical_features, gossipcop_new_features])

In [49]:
historical_features["news_lifetime"] = historical_features["news_lifetime"].astype(np.int64)

In [50]:
historical_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, 0 to 28
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    562 non-null    object 
 1   average_followers          562 non-null    float64
 2   average_follows            562 non-null    float64
 3   repost_total               562 non-null    int64  
 4   post_total                 562 non-null    int64  
 5   repost_percentage          562 non-null    float64
 6   average_repost             562 non-null    float64
 7   average_favorite           562 non-null    float64
 8   label                      562 non-null    int64  
 9   news_lifetime              562 non-null    int64  
 10  nb_users_10_hours          562 non-null    int64  
 11  average_time_difference    562 non-null    float64
 12  retweet_percentage_1_hour  562 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 61.5+ K

In [51]:
historical_features.groupby("label").size()

label
0    277
1    285
dtype: int64

In [52]:
historical_features.columns

Index(['news_id', 'average_followers', 'average_follows', 'repost_total',
       'post_total', 'repost_percentage', 'average_repost', 'average_favorite',
       'label', 'news_lifetime', 'nb_users_10_hours',
       'average_time_difference', 'retweet_percentage_1_hour'],
      dtype='object')

In [53]:
historical_features.to_csv("data/historical_features.csv", index=False)

Save all the data into a Feature Store via Hopsworks

Insert dataframe into Feature Group, ```expectation_suite``` should contain data validation rules => we need to come up withd data validation rules

Do we actually need an event time ?

In [54]:
news_propagation_fg = fs.get_or_create_feature_group(
    name='news_propagation',
    description='Propagation characteristics of news in Bsky',
    version=1,
    primary_key=['news_id'],
    online_enabled=True,
)

In [55]:
news_propagation_fg.insert(historical_features, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 562/562 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: news_propagation_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1149079/jobs/named/news_propagation_1_offline_fg_materialization/executions
2025-01-07 20:55:17,092 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-07 20:55:20,291 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-07 20:56:40,583 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-07 20:56:40,753 INFO: Waiting for log aggregation to finish.
2025-01-07 20:56:49,388 INFO: Execution finished successfully.


(Job('news_propagation_1_offline_fg_materialization', 'SPARK'), None)

Add a description for each feature in the feature group.

In [56]:
news_propagation_fg.update_feature_description("news_id", "The id of the news")
news_propagation_fg.update_feature_description("average_followers", "The average number of followers of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("average_follows", "The average number of follows of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("repost_total", "The total number of reposts of the news")
news_propagation_fg.update_feature_description("post_total", "The total number of posts of the news")
news_propagation_fg.update_feature_description("repost_percentage", "The percentage of reposts of the news out of the total number of posts and reposts")
news_propagation_fg.update_feature_description("average_repost", "The average number of reposts of the news")
news_propagation_fg.update_feature_description("average_favorite", "The average number of likes of the news")
news_propagation_fg.update_feature_description("label", "The label of the news : if it is fake or real")
news_propagation_fg.update_feature_description("news_lifetime", "The lifetime of the news, which is the time difference between the first and the last post (or repost) of the news")
news_propagation_fg.update_feature_description("nb_users_10_hours", "The number of users who posted (or reposted) the news in the first 10 hours")
news_propagation_fg.update_feature_description("average_time_difference", "The average time difference between a post and its reposts of the news")
news_propagation_fg.update_feature_description("retweet_percentage_1_hour", "The percentage of retweets of the news in the first hour")
